In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import re

def extract_election_data(html_path):
    with open(html_path, 'r', encoding='utf-8') as source:
        content = source.read()
    
    parsed_html = BeautifulSoup(content, 'html.parser')
    
    # Extract party-wise results
    result_grid = parsed_html.find('table', class_='table')
    entries = result_grid.find_all('tr')[1:-1]  # Skip header and footer
    
    party_data = []
    for entry in entries:
        cells = entry.find_all('td')
        if len(cells) == 4:
            party_data.append({
                'Political Party': cells[0].text.strip(),
                'Seats Won': int(cells[1].text.strip()),
                'Seats Leading': int(cells[2].text.strip()),
                'Total Seats': int(cells[3].text.strip())
            })
    
    # Extract vote share information
    vote_info = []
    scripts = parsed_html.find_all('script')
    for script in scripts:
        script_content = script.string
        if script_content and all(var in script_content for var in ['var xValues', 'var yValues']):
            party_names = re.search(r'var xValues = \[(.*?)\];', script_content, re.DOTALL)
            vote_counts = re.search(r'var yValues = \[(.*?)\];', script_content, re.DOTALL)
            
            if party_names and vote_counts:
                parties = eval('[' + party_names.group(1) + ']')
                votes = eval('[' + vote_counts.group(1) + ']')
                
                for party, vote in zip(parties, votes):
                    parsed = re.match(r'([^{]+)(?:\{([^}]+)\})?', party)
                    if parsed:
                        party_name = parsed.group(1).strip()
                        share = parsed.group(2)
                        share_percentage = float(share.strip('%')) if share else None
                        vote_info.append({
                            'Political Party': party_name,
                            'Vote Percentage': share_percentage,
                            'Vote Count': vote
                        })
    
    return pd.DataFrame(party_data), pd.DataFrame(vote_info)

# Main execution
if __name__ == "__main__":
    html_file = 'election_results.html'
    party_results, vote_distribution = extract_election_data(html_file)
    
    print("Election Results by Party:")
    print(party_results)
    print("\nVote Distribution:")
    print(vote_distribution)
    
    # Save to CSV files
    party_results.to_csv('election_party_outcomes.csv', index=False)
    vote_distribution.to_csv('election_vote_statistics.csv', index=False)

Election Results by Party:
                                      Political Party  Seats Won  \
0                        Bharatiya Janata Party - BJP        240   
1                      Indian National Congress - INC         99   
2                                Samajwadi Party - SP         37   
3                 All India Trinamool Congress - AITC         29   
4                     Dravida Munnetra Kazhagam - DMK         22   
5                                  Telugu Desam - TDP         16   
6                        Janata Dal  (United) - JD(U)         12   
7      Shiv Sena (Uddhav Balasaheb Thackrey) - SHSUBT          9   
8   Nationalist Congress Party – Sharadchandra Paw...          8   
9                                     Shiv Sena - SHS          7   
10             Lok Janshakti Party(Ram Vilas) - LJPRV          5   
11      Yuvajana Sramika Rythu Congress Party - YSRCP          4   
12                         Rashtriya Janata Dal - RJD          4   
13       Communist Pa